In [1]:
import os
import time
import pyexiv2
import subprocess
import numpy as np
from tqdm import tqdm

In [5]:
# Get all notebook paths from the matplotlib gallery
# NOTE: excludes animation examples as they aren't currently supporded by matplotalt
notebook_paths = []
notebook_names = []
gallery_path = "./gallery_jupyter"
for notebook_dir in os.listdir(gallery_path):
    notebook_dir_path = os.path.join(gallery_path, notebook_dir)
    if os.path.isdir(notebook_dir_path):
        for notebook_name in os.listdir(notebook_dir_path):
            notebook_path = os.path.join(notebook_dir_path, notebook_name)
            if os.path.isfile(notebook_path) and notebook_name.endswith(".ipynb"):
                notebook_paths.append(notebook_path)
                notebook_names.append(notebook_name)
print(len(notebook_paths))

500


In [7]:
# Run alttextify on each matplotlib example notebook
notebook_output_path = "./gallery_with_alt"
fig_output_path = "./alt_figs"
nb_execution_times = []
for nb_idx, nb_path in enumerate(tqdm(notebook_paths)):
    start = time.time()
    subprocess.run(["alttextify", nb_path,
                   f"{notebook_output_path}/nb_{nb_idx}_{notebook_names[nb_idx]}",
                   "-t", "30",
                   "-s", "html", "img_file",
                   "-so", f"{fig_output_path}/nb_{nb_idx}"])
    end = time.time()
    nb_execution_times.append(end - start)

print(f"Total execution time: {np.sum(nb_execution_times)}")
print(f"Mean execution time: {np.mean(nb_execution_times)}")
np.save("./nb_execution_times", nb_execution_times)

 40%|████      | 202/500 [29:16<43:11,  8.70s/it] 


KeyboardInterrupt: 

In [14]:
# Extract image descriptions from figures
for fig_with_caption in os.listdir(fig_output_path):
    fig_path = os.path.join(fig_output_path, fig_with_caption)
    with open(fig_path, 'rb+') as img_f:
        with pyexiv2.ImageData(img_f.read()) as pyexif_img:
            print(pyexif_img.read_exif()['Exif.Image.ImageDescription'])

./alt_figs\nb_0.jpg
A 10x10 image. The x-axis ranges from −2 to 10, the y-axis ranges from −2 to 10, and the z-axis ranges from 0.03307 to 0.9778, all using linear scales. The data have a minimum value of z=0.03307 at (9, 3), a maximum value of z=0.9778 at (0, 4), and an average of z=0.5512.
./alt_figs\nb_1.jpg
A group of 5 subplots subplot 0: a 15x15 image. The x-axis ranges from −4 to 4, the y-axis ranges from −4 to 3, and the z-axis ranges from -1.694 to 1.386, all using linear scales. The data have a minimum value of z=-1.694 at (10, 9), a maximum value of z=1.386 at (7, 6), and an average of z=0.00283. subplot 1: a 15x15 image. The x-axis ranges from −3 to 4, the y-axis ranges from −4 to 3, and the z-axis ranges from -1.694 to 1.386, all using linear scales. The data have a minimum value of z=-1.694 at (10, 9), a maximum value of z=1.386 at (7, 6), and an average of z=0.00283. subplot 2: a 15x15 image. The x-axis ranges from −4 to 4, the y-axis ranges from −4 to 3, and the z-axis 